In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import sys
import cv2 as cv
from tqdm import tqdm

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
combine_fold = 'C:/Users/oryan/Documents/mergers-in-cosmos/cats-to-combine'
data_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/back-to-basics-data'
fig_folder = 'C:/Users/oryan/Documents/mergers-in-cosmos/paper-source/figures'
cos_dat_fold = 'E:/cosmos-data'

In [3]:
df_cat = pd.read_csv(f'{combine_fold}/dor-all-restaged-full.csv', index_col = 0)

C:\Users\oryan\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_cat

,ORyanID_23,ID,category,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,stage4,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,...,-8.931825,-8.911355,-8.899668,-8.883307,-8.872990,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705533383,860686,stage3,150.644522,2.237226,10047.978516,22713.351562,0.000051,0.000046,1.571297e-06,...,-9.314209,-9.106611,-8.941274,-8.713107,-8.541205,0.341270,0.545498,0.732327,0.906665,1.020098
2,4000705539941,1301188,stage2,149.702462,2.654895,32637.949219,32737.013672,0.000064,0.000104,-2.881107e-06,...,-9.794128,-9.784252,-9.766013,-9.743796,-9.727879,0.018737,0.018737,0.018737,0.018737,0.018737
3,4000705540034,1304538,stage3,149.726828,2.658172,32053.712891,32815.468750,0.000071,0.000086,-6.036952e-06,...,-9.480532,-9.443615,-9.402433,-9.352081,-9.325659,0.302629,0.376027,0.440394,0.508153,0.567727
4,4000705540064,1310001,stage3,149.701230,2.660343,32667.441406,32867.789062,0.000026,0.000020,-1.023530e-05,...,-9.882174,-9.855921,-9.816085,-9.697225,-9.573506,0.455258,0.482555,0.522239,0.600383,0.660279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4176,4001204758614_extra,1520733,stage4,149.891374,2.866981,28107.876953,37826.437500,0.000086,0.000104,-9.313976e-07,...,-9.441206,-9.279022,-9.095134,-9.003501,-8.955496,0.120441,0.213251,0.306832,0.556012,0.731876
4177,4546055995_extra,1520749,stage2,150.383110,2.865367,16320.103516,37787.917969,0.000131,0.000236,7.123858e-05,...,-9.857880,-9.658215,-9.485625,-9.235816,-8.971795,0.162147,0.238640,0.432004,0.592193,0.687256
4178,4000927318133_extra,1523385,stage3,149.789910,2.865132,30540.179688,37782.648438,0.000045,0.000059,1.600747e-05,...,-9.525900,-9.472779,-9.353454,-9.273221,-9.213521,0.262676,0.355466,0.410356,0.489105,0.566003
4179,4000747939904_extra,1524909,stage3,150.450581,2.872371,14702.710938,37956.468750,0.000248,0.000292,7.378556e-06,...,-9.556987,-9.385500,-9.154130,-9.026810,-8.938077,0.153294,0.283249,0.436332,0.600615,0.693621


In [5]:
df_red_tmp = df_cat.query('ez_sfr < 10')
df_red = df_red_tmp.query('lp_mass_best > 6.5')

### Cross Matching with AGN Catalogues

In [6]:
df_red

,ORyanID_23,ID,category,ALPHA_J2000,DELTA_J2000,X_IMAGE,Y_IMAGE,ERRX2_IMAGE,ERRY2_IMAGE,ERRXY_IMAGE,...,ez_ssfr_p025,ez_ssfr_p160,ez_ssfr_p500,ez_ssfr_p840,ez_ssfr_p975,ez_Av_p025,ez_Av_p160,ez_Av_p500,ez_Av_p840,ez_Av_p975
0,4000705532455,816891,stage4,150.679846,2.196543,9200.399414,21737.232422,0.000052,0.000059,-1.043282e-05,...,-8.931825,-8.911355,-8.899668,-8.883307,-8.872990,0.529107,0.560142,0.599624,0.630920,0.652330
1,4000705533383,860686,stage3,150.644522,2.237226,10047.978516,22713.351562,0.000051,0.000046,1.571297e-06,...,-9.314209,-9.106611,-8.941274,-8.713107,-8.541205,0.341270,0.545498,0.732327,0.906665,1.020098
2,4000705539941,1301188,stage2,149.702462,2.654895,32637.949219,32737.013672,0.000064,0.000104,-2.881107e-06,...,-9.794128,-9.784252,-9.766013,-9.743796,-9.727879,0.018737,0.018737,0.018737,0.018737,0.018737
3,4000705540034,1304538,stage3,149.726828,2.658172,32053.712891,32815.468750,0.000071,0.000086,-6.036952e-06,...,-9.480532,-9.443615,-9.402433,-9.352081,-9.325659,0.302629,0.376027,0.440394,0.508153,0.567727
4,4000705540064,1310001,stage3,149.701230,2.660343,32667.441406,32867.789062,0.000026,0.000020,-1.023530e-05,...,-9.882174,-9.855921,-9.816085,-9.697225,-9.573506,0.455258,0.482555,0.522239,0.600383,0.660279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4176,4001204758614_extra,1520733,stage4,149.891374,2.866981,28107.876953,37826.437500,0.000086,0.000104,-9.313976e-07,...,-9.441206,-9.279022,-9.095134,-9.003501,-8.955496,0.120441,0.213251,0.306832,0.556012,0.731876
4177,4546055995_extra,1520749,stage2,150.383110,2.865367,16320.103516,37787.917969,0.000131,0.000236,7.123858e-05,...,-9.857880,-9.658215,-9.485625,-9.235816,-8.971795,0.162147,0.238640,0.432004,0.592193,0.687256
4178,4000927318133_extra,1523385,stage3,149.789910,2.865132,30540.179688,37782.648438,0.000045,0.000059,1.600747e-05,...,-9.525900,-9.472779,-9.353454,-9.273221,-9.213521,0.262676,0.355466,0.410356,0.489105,0.566003
4179,4000747939904_extra,1524909,stage3,150.450581,2.872371,14702.710938,37956.468750,0.000248,0.000292,7.378556e-06,...,-9.556987,-9.385500,-9.154130,-9.026810,-8.938077,0.153294,0.283249,0.436332,0.600615,0.693621


In [7]:
df_coords = (
    df_red[['ID', 'ALPHA_J2000', 'DELTA_J2000']]
    .rename(columns = {'ALPHA_J2000':'RA','DELTA_J2000':'DEC'})
    .set_index('ID')
)

In [8]:
coords_dict = df_coords.to_dict(orient='index')

### Smolcic Catalogue 

In [9]:
with fits.open(f'{cos_dat_fold}/VLA_3GHz_counterpart_array_20170210_paper_smolcic_et_al.fits') as hdul:
    smol_rec = hdul[1].data

In [10]:
smol_rec.names

['ID_VLA',
 'RA_VLA_J2000',
 'DEC_VLA_J2000',
 'MULTI',
 'CAT_CPT',
 'ID_CPT',
 'RA_CPT_J2000',
 'DEC_CPT_J2000',
 'SEP_VLA_CPT',
 'P_FALSE',
 'Z_BEST',
 'FLUX_INT_3GHz',
 'Lradio_10cm',
 'Lradio_21cm',
 'L_TIR_SF',
 'SFR_KENN98',
 'Xray_AGN',
 'MIR_AGN',
 'SED_AGN',
 'Quiescent_MLAGN',
 'SFG',
 'Clean_SFG',
 'HLAGN',
 'MLAGN',
 'Radio_excess',
 'flag_COSMOS2015']

In [11]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_smolcic_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_VLA_J2000'] * u.deg, dec = tab['DEC_VLA_J2000'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'smol_ID' : tab[min_ind]['ID_VLA'], 'RA' : tab[min_ind]['RA_VLA_J2000'], 'Dec' : tab[min_ind]['DEC_VLA_J2000']}

In [12]:
def matching_smolcic_ids(ra, dec, smol_rec):
    arcsec_sep_deg = 15 / (60 * 60)
    
    matching_tab = smol_rec[(smol_rec['RA_VLA_J2000'] > ra - arcsec_sep_deg) & (smol_rec['RA_VLA_J2000'] < ra + arcsec_sep_deg) & (smol_rec['DEC_VLA_J2000'] > dec - arcsec_sep_deg) & (smol_rec['DEC_VLA_J2000'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'smol_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'smol_ID' : matching_tab['ID_VLA'][0], 'RA' : matching_tab['RA_VLA_J2000'][0], 'Dec' : matching_tab['DEC_VLA_J2000'][0]}
    elif len(matching_tab) > 1:
        return get_correct_smolcic_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [13]:
smolcic_matched_dict = {}

In [14]:
done_ids = list(smolcic_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    smolcic_matched_dict[i] = matching_smolcic_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], smol_rec)

100%|██████████| 4152/4152 [00:36<00:00, 114.73it/s]


In [15]:
smolcic_matched_df = pd.DataFrame.from_dict(smolcic_matched_dict, orient = 'index').dropna().reset_index().rename(columns = {'index' : 'ID', 0 : 'smol_id'})

In [16]:
smolcic_matched_df

,ID,smol_ID,RA,Dec
0,1261404,3714.0,150.279358,2.609581
1,1264241,1766.0,150.321804,2.617930
2,844303,350.0,150.059144,2.219995
3,219965,3890.0,150.224529,1.618718
4,230625,10131.0,150.231835,1.627988
...,...,...,...,...
1812,1519437,2509.0,150.151975,2.860138
1813,1519536,1517.0,150.380343,2.865278
1814,1520749,1517.0,150.380343,2.865278
1815,1524909,8321.0,150.448344,2.875046


In [17]:
def dedup_smol(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return int(min_row.ID.iloc[0])

In [18]:
df_smol_merg = (
    smolcic_matched_df
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'smol_ID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [19]:
smol_dedup_df = pd.DataFrame(df_smol_merg.groupby('smol_ID').apply(lambda df: dedup_smol(df))).reset_index().rename(columns = {0 : 'ID'})

In [20]:
smol_dedup_df

,smol_ID,ID
0,8.0,888234
1,25.0,1086305
2,47.0,404744
3,61.0,835729
4,66.0,1152835
...,...,...
1130,10866.0,944937
1131,10869.0,1341925
1132,10900.0,1368259
1133,10912.0,893170


In [21]:
def getting_smol_info(smol_id, smol_rec):
    row = smol_rec[smol_rec['ID_VLA'] == smol_id]
    clsfs = [row['SFG'][0], row['Clean_SFG'][0], row['Radio_excess'][0]]
    
    if clsfs[0] == 'true' and clsfs[1] == 'true':
        return 'SFG'
    elif clsfs[0] == 'false' and clsfs[1] == 'false':
        return 'AGN'
    elif clsfs[0] == 'true' and clsfs[1] == 'false':
        return 'SFG'
    elif clsfs[0] == 'false' and clsfs[1] == 'true':
        print('Something wrong: 1')
        sys.exit()
    elif clsfs[3] == 'true':
        return 'Radio_Excess'

In [22]:
smol_clsf_dict = {}

In [23]:
smol_dedup_dict = smol_dedup_df.set_index('ID').to_dict(orient = 'index')

In [24]:
done_ids = list(smol_clsf_dict.keys())
for i in tqdm(list(smol_dedup_dict.keys())):
    if i in done_ids:
        continue
    
    smol_clsf_dict[i] = getting_smol_info(smol_dedup_dict[i]['smol_ID'], smol_rec)

100%|██████████| 1135/1135 [00:08<00:00, 127.25it/s]


In [25]:
smol_clsf_df = pd.DataFrame.from_dict(smol_clsf_dict, orient = 'index').reset_index().rename(columns = {'index' : 'ID', 0 : 'agn_clsf'})

In [26]:
len(smol_clsf_df)

1135

In [27]:
smol_clsf_df.drop_duplicates(subset='ID', keep='first')

,ID,agn_clsf
0,888234,AGN
1,1086305,AGN
2,404744,AGN
3,835729,AGN
4,1152835,AGN
...,...,...
1130,944937,SFG
1131,1341925,SFG
1132,1368259,AGN
1133,893170,AGN


In [28]:
smol_clsf_df.agn_clsf.value_counts()

SFG    761
AGN    374
Name: agn_clsf, dtype: int64

In [29]:
smol_cat_ids = list(smol_clsf_df.ID)

### The VLA 3GHz Counterpart Array

In [30]:
with fits.open(f'{cos_dat_fold}/VLA_3GHz_counterpart_array_20170210_paper_delvecchio_et_al.fits') as hdul:
    vla_cat = hdul[1].data

In [31]:
vla_cat.names

['ID_VLA3',
 'RA_VLA3',
 'DEC_VLA3',
 'Z_BEST',
 'Z_TYPE',
 'FLUX_INT_3GHz',
 'Lradio_10cm',
 'Lradio_21cm',
 'L_TIR_SF',
 'flag_herschel3',
 'Mstar',
 'SFR_IR',
 'NUV_R',
 'XRAY_AGN',
 'MIR_AGN',
 'SED_AGN',
 'Radio_excess',
 'Class']

In [32]:
len(vla_cat)

7729

In [33]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_vla_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA_VLA3'] * u.deg, dec = tab['DEC_VLA3'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'VLA_ID' : tab[min_ind]['ID_VLA3'], 'RA' : tab[min_ind]['RA_VLA3'], 'Dec' : tab[min_ind]['DEC_VLA3']}

In [34]:
def matching_vla_ids(ra, dec, vla_cat):
    arcsec_sep_deg = 15 / (60 * 60)
    
    matching_tab = vla_cat[(vla_cat['RA_VLA3'] > ra - arcsec_sep_deg) & (vla_cat['RA_VLA3'] < ra + arcsec_sep_deg) & (vla_cat['DEC_VLA3'] > dec - arcsec_sep_deg) & (vla_cat['DEC_VLA3'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'VLA_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'VLA_ID' : matching_tab['ID_VLA3'][0], 'RA' : matching_tab['RA_VLA3'][0], 'Dec' : matching_tab['DEC_VLA3'][0]}
    elif len(matching_tab) > 1:
        return get_correct_vla_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [35]:
vla_matched_dict = {}

In [36]:
done_ids = list(vla_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    vla_matched_dict[i] = matching_vla_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], vla_cat)

100%|██████████| 4152/4152 [00:26<00:00, 155.16it/s]


In [37]:
vla_matched_df = (
    pd.DataFrame.from_dict(vla_matched_dict, orient = 'index')
    .dropna()
    .reset_index()
    .rename(columns = {'index':'ID', 0 : 'VLA_ID_arr'})
)

In [38]:
vla_matched_df

,ID,VLA_ID,RA,Dec
0,1261404,3714.0,150.279358,2.609581
1,1264241,1766.0,150.321804,2.617930
2,844303,350.0,150.059144,2.219995
3,246486,8866.0,150.233316,1.651868
4,250320,8866.0,150.233316,1.651868
...,...,...,...,...
1658,1519437,2509.0,150.151975,2.860138
1659,1519536,1517.0,150.380343,2.865278
1660,1520749,1517.0,150.380343,2.865278
1661,1524909,8321.0,150.448344,2.875046


In [39]:
def dedup_vla(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return int(min_row.ID.iloc[0])

In [40]:
df_vla_merg = (
    vla_matched_df
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'VLA_ID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [41]:
vla_dedup_df = pd.DataFrame(df_vla_merg.groupby('VLA_ID').apply(lambda df: dedup_vla(df))).reset_index().rename(columns = {0 : 'ID'})

In [42]:
len(vla_dedup_df)

1037

In [43]:
vla_checked = (
    vla_dedup_df
    .assign(in_smol = vla_dedup_df.ID.apply(lambda x: 'y' if x in smol_cat_ids else 'n'))
)

vla_checked_df = vla_checked.query('in_smol == "n"').drop(columns = 'in_smol')

In [44]:
len(vla_checked_df)

4

In [45]:
vla_dict = vla_checked_df.set_index('ID').to_dict(orient = 'index')

In [46]:
vla_cat_dict = {}

In [47]:
def getting_vla_info(vla_id, vla_cat):
    row = vla_cat[vla_cat['ID_VLA3'] == vla_id]
    ans = row['CLASS'][0]
    
    if len(ans) < 0.5:
        return 'SFG'
    else:
        return ans

In [48]:
done_ids = list(vla_cat_dict.keys())
for i in tqdm(list(vla_dict.keys())):
    if i in done_ids:
        continue
    
    vla_cat_dict[i] = getting_vla_info(vla_dict[i]['VLA_ID'], vla_cat)

100%|██████████| 4/4 [00:00<00:00, 144.39it/s]


In [49]:
vla_cat_df = pd.DataFrame.from_dict(vla_cat_dict, orient = 'index').rename(columns = {'index' : 'ID', 0 : 'clsf'}).dropna()
vla_cat_df

,clsf
765389,SFG
715417,HLAGN
1485262,SFG
998663,HLAGN


In [50]:
vla_cat_df.clsf.value_counts()

SFG      2
HLAGN    2
Name: clsf, dtype: int64

In [51]:
vla_cat_ids = list(vla_cat_df.reset_index()['index'])

### Applying to Chandra

In [52]:
with fits.open(f'{cos_dat_fold}/chandra_COSMOS_legacy_opt_NIR_counterparts_20160113_4d.fits') as hdul:
    chandra_cat = hdul[1].data

In [53]:
chandra_cat.names

['id_x',
 'RA_x',
 'DEC_x',
 'DET_ML_F',
 'DET_ML_S',
 'DET_ML_H',
 'flux_F',
 'flux_S',
 'flux_H',
 'HR',
 'HR_lo_lim',
 'HR_up_lim',
 'id_i',
 'ra_i',
 'dec_i',
 'i_mag',
 'i_mag_err',
 'flag_i_mag',
 'id_k_uv',
 'ra_k_uv',
 'dec_k_uv',
 'k_uv_mag',
 'k_uv_mag_err',
 'id_k_cfht',
 'ra_k_cfht',
 'dec_k_cfht',
 'k_cfht_mag',
 'k_cfht_mag_err',
 'ra_irac',
 'dec_irac',
 'irac_mag',
 'irac_mag_err',
 'ra_splash',
 'dec_splash',
 'splash_mag',
 'splash_mag_err',
 'final_id_flag',
 'star_flag',
 'z_best',
 'z_spec',
 'Instr',
 'Qg',
 'spec_type',
 'z_phot',
 'phot_type',
 'id_xmm',
 'dist_lum',
 'Lx_0510',
 'Lx_052',
 'Lx_210',
 'NH',
 'abs_corr_0510',
 'abs_corr_052',
 'abs_corr_210',
 'NH_lo',
 'abs_corr_0510_lo',
 'abs_corr_052_lo',
 'abs_corr_210_lo',
 'NH_up',
 'abs_corr_0510_up',
 'abs_corr_052_up',
 'abs_corr_210_up']

In [54]:
len(chandra_cat)

4016

In [55]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_chandra_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    chandra_coords = SkyCoord(ra = tab['RA_x'] * u.deg, dec = tab['DEC_x'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(chandra_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'CHANDRA_ID' : tab[min_ind]['id_x'], 'RA' : tab[min_ind]['RA_x'], 'Dec' : tab[min_ind]['DEC_x']}

In [56]:
def matching_chandra_ids(ra, dec, chandra_cat):
    arcsec_sep_deg = 15 / (60 * 60)
    
    matching_tab = chandra_cat[(chandra_cat['RA_x'] > ra - arcsec_sep_deg) & (chandra_cat['RA_x'] < ra + arcsec_sep_deg) & (chandra_cat['DEC_x'] > dec - arcsec_sep_deg) & (chandra_cat['DEC_x'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'CHANDRA_ID' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'CHANDRA_ID' : matching_tab['id_x'][0], 'RA' : matching_tab['RA_x'][0], 'Dec' : matching_tab['DEC_x'][0]}
    elif len(matching_tab) > 1:
        return get_correct_chandra_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [57]:
chandra_matched_dict = {}

In [58]:
done_ids = list(chandra_matched_dict.keys())
for i in tqdm(list(coords_dict.keys())):
    if i in done_ids:
        continue
    
    chandra_matched_dict[i] = matching_chandra_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], chandra_cat)

100%|██████████| 4152/4152 [01:14<00:00, 55.75it/s]


In [59]:
chandra_matched_df = (
    pd.DataFrame.from_dict(chandra_matched_dict, orient = 'index')
    .dropna()
    .reset_index()
    .rename(columns = {'index':'ID'})
)

In [60]:
chandra_matched_df

,ID,CHANDRA_ID,RA,Dec
0,1264241,lid_2035,150.320007,2.620354
1,844303,cid_1139,150.059130,2.219824
2,230625,cid_737,150.231813,1.628142
3,525757,cid_236,150.027300,1.924484
4,1459748,lid_487,150.672224,2.811132
...,...,...,...,...
743,1464904,lid_3181,150.379151,2.817332
744,1465158,lid_3224,150.673890,2.814361
745,1482513,lid_3248,149.951376,2.823183
746,1500329,lid_578,149.888987,2.843867


In [61]:
def dedup_chandra(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return int(min_row.ID.iloc[0])

In [62]:
df_chandra_merg = (
    chandra_matched_df
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'CHANDRA_ID', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [63]:
chandra_dedup_df = pd.DataFrame(df_chandra_merg.groupby('CHANDRA_ID').apply(lambda df: dedup_chandra(df))).reset_index().rename(columns = {0 : 'ID'})

In [64]:
len(chandra_dedup_df)

448

In [65]:
chandra_checked_df = (
    chandra_dedup_df
    .assign(in_cmol = chandra_dedup_df.ID.apply(lambda x: 'y' if x in smol_cat_ids else 'n'))
    .assign(in_vla = chandra_dedup_df.ID.apply(lambda x: 'y' if x in vla_cat_ids else 'n'))
)

In [66]:
len(chandra_checked_df)

448

In [67]:
chandra_mpa_checked = chandra_checked_df.query('in_vla == "n" and in_cmol == "n"')

In [68]:
len(chandra_mpa_checked)

221

In [69]:
chandra_dict = chandra_mpa_checked.set_index('ID').to_dict(orient = 'index')

In [76]:
chandra_cat_dict = {}

In [83]:
def getting_chandra_info(chandra_id, chandra_cat):
    row = chandra_cat[chandra_cat['id_x'] == chandra_id]    
    hard_lum = row['Lx_210'][0]
    
    if hard_lum < 42:
        return 'SFG'
    elif hard_lum >= 42:
        return 'AGN'
    else:
        return np.nan

In [84]:
done_ids = list(chandra_cat_dict.keys())
for i in tqdm(list(chandra_dict.keys())):
    if i in done_ids:
        continue
    
    chandra_cat_dict[i] = getting_chandra_info(chandra_dict[i]['CHANDRA_ID'], chandra_cat)

100%|██████████| 221/221 [00:03<00:00, 60.07it/s]


In [85]:
chandra_cat_df = pd.DataFrame.from_dict(chandra_cat_dict, orient = 'index').rename(columns = {'index' : 'ID', 0 : 'clsf'}).dropna()
chandra_cat_df

,clsf
352373,AGN
418853,SFG
788036,AGN
1120198,SFG
633485,SFG
...,...
1470183,AGN
938996,AGN
500953,AGN
410793,AGN


In [87]:
chandra_cat_df.reset_index().rename(columns = {'index' : 'ID'}).drop_duplicates(subset='ID', keep='first')

,ID,clsf
0,352373,AGN
1,418853,SFG
2,788036,AGN
3,1120198,SFG
4,633485,SFG
...,...,...
216,1470183,AGN
217,938996,AGN
218,500953,AGN
219,410793,AGN


In [88]:
chandra_cat_df.clsf.value_counts()

AGN    158
SFG     63
Name: clsf, dtype: int64

### Finally, Looking at MPA-JHU

In [89]:
agn_cat = pd.concat(
    [
        smol_clsf_df, 
        vla_cat_df.reset_index().rename(columns = {'index' : 'ID', 'clsf' : 'agn_clsf'}), 
        chandra_cat_df.reset_index().rename(columns = {'index' : 'ID', 'clsf' : 'agn_clsf'})
    ]
)

In [90]:
len(agn_cat)

1360

In [91]:
agn_cat.drop_duplicates(subset='ID', keep='first')

,ID,agn_clsf
0,888234,AGN
1,1086305,AGN
2,404744,AGN
3,835729,AGN
4,1152835,AGN
...,...,...
216,1470183,AGN
217,938996,AGN
218,500953,AGN
219,410793,AGN


In [92]:
agn_cat.agn_clsf.value_counts()

SFG      826
AGN      532
HLAGN      2
Name: agn_clsf, dtype: int64

In [93]:
found_ids = list(agn_cat.ID)

In [94]:
with fits.open(f'{cos_dat_fold}/MPA-JHU_full_catalogue.fits') as hdul:
    mpa_jhu = hdul[1].data

In [95]:
# Function that selects the closest object in the MPA_JHU as the galaxy of interest.
def get_correct_mpa_id(ra, dec, tab):
    cosmos_coord = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    mpa_coords = SkyCoord(ra = tab['RA'] * u.deg, dec = tab['DEC'] * u.deg, frame = 'fk5')
    
    seps = list(abs((cosmos_coord.separation(mpa_coords)).to(u.arcsec) / u.arcsec))
    
    min_ind = seps.index(min(seps))
    
    return {'SPECOBJID_1' : tab[min_ind]['SPECOBJID_1'], 'RA' : tab[min_ind]['RA'], 'Dec' : tab[min_ind]['DEC']}

In [96]:
def matching_mpa_ids(ra, dec, mpa_cat):
    arcsec_sep_deg = 15 / (60 * 60)
    
    matching_tab = mpa_cat[(mpa_cat['RA'] > ra - arcsec_sep_deg) & (mpa_cat['RA'] < ra + arcsec_sep_deg) & (mpa_cat['DEC'] > dec - arcsec_sep_deg) & (mpa_cat['DEC'] < dec + arcsec_sep_deg)]
    if len(matching_tab) < 0.5:
        return {'SPECOBJID_1' : np.nan, 'RA' : np.nan, 'Dec' : np.nan}
    elif len(matching_tab) == 1:
        return {'SPECOBJID_1' : matching_tab['SPECOBJID_1'][0], 'RA' : matching_tab['RA'][0], 'Dec' : matching_tab['DEC'][0]}
    elif len(matching_tab) > 1:
        return get_correct_mpa_id(ra, dec, matching_tab)
    else:
        print('Somethings gone very wrong!!')
        sys.exit()

In [97]:
mpa_matched_dict = {}

In [98]:
for i in tqdm(list(coords_dict.keys())):
    if i in found_ids:
        continue
    
    mpa_matched_dict[i] = matching_mpa_ids(coords_dict[i]['RA'], coords_dict[i]['DEC'], mpa_jhu)
    
    found_ids.append(i)

100%|██████████| 4152/4152 [18:02<00:00,  3.83it/s]


In [99]:
mpa_matched_df = (
    pd.DataFrame.from_dict(mpa_matched_dict, orient = 'index')
    .dropna()
    .reset_index()
    .rename(columns = {'index':'ID'})
)

In [100]:
mpa_matched_df

,ID,SPECOBJID_1,RA,Dec
0,1156381,563113264351832064,150.403885,2.508875
1,578325,564139662499670016,150.522308,1.974735
2,1473567,563107766793693184,149.934113,2.820669
3,724534,564181993697339392,150.179733,2.110343
4,1084278,564180894185711616,150.377701,2.440064
5,1121040,563101169723926528,150.185944,2.474567
6,1431797,563117662398343168,150.090851,2.782693
7,241559,562994242218125312,149.761536,1.646707
8,1018703,564172922726410240,150.027725,2.373884
9,858969,563089349973927936,149.788071,2.234646


In [101]:
def dedup_mpa(df):
    
    ra = df.RA.iloc[0]
    dec = df.Dec.iloc[0]
    
    if len(df) == 1:
        return df.ID.iloc[0]
    
    coords = SkyCoord(ra = ra * u.deg, dec = dec * u.deg, frame = 'fk5')
    
    df_coords = df.assign(coords = df.apply(lambda row: SkyCoord(ra = row.ALPHA_J2000 * u.deg, dec = row.DELTA_J2000 * u.deg, frame = 'fk5'), axis = 1))
    
    df_seps = df_coords.assign(seps = df_coords.coords.apply(lambda x: np.float64(abs(((x.separation(coords)).to(u.arcsec) / u.arcsec)))))
    min_sep = df_seps.seps.min()
    min_row = df_seps.query('seps == @min_sep')
    return int(min_row.ID.iloc[0])

In [102]:
df_mpa_merg = (
    mpa_matched_df
    .merge(df_cat, on = 'ID', how = 'left')
    [['ID', 'SPECOBJID_1', 'RA', 'Dec', 'ALPHA_J2000', 'DELTA_J2000']]
)

In [103]:
df_mpa_merg

,ID,SPECOBJID_1,RA,Dec,ALPHA_J2000,DELTA_J2000
0,1156381,563113264351832064,150.403885,2.508875,150.407974,2.507811
1,578325,564139662499670016,150.522308,1.974735,150.525207,1.974372
2,1473567,563107766793693184,149.934113,2.820669,149.931932,2.824748
3,724534,564181993697339392,150.179733,2.110343,150.176286,2.111888
4,1084278,564180894185711616,150.377701,2.440064,150.375383,2.441677
5,1121040,563101169723926528,150.185944,2.474567,150.186763,2.478538
6,1431797,563117662398343168,150.090851,2.782693,150.087899,2.779759
7,241559,562994242218125312,149.761536,1.646707,149.764582,1.644018
8,1018703,564172922726410240,150.027725,2.373884,150.028460,2.375080
9,858969,563089349973927936,149.788071,2.234646,149.786440,2.234091


In [104]:
mpa_dedup_df = pd.DataFrame(df_mpa_merg.groupby('SPECOBJID_1').apply(lambda df: dedup_mpa(df))).reset_index().rename(columns = {0 : 'ID'})

In [105]:
mpa_dedup_df

,SPECOBJID_1,ID
0,562994242218125312,241559
1,563089349973927936,858969
2,563101169723926528,1121040
3,563107766793693184,1473567
4,563113264351832064,1156381
5,563117662398343168,1431797
6,564139662499670016,576003
7,564172922726410240,1018703
8,564180894185711616,1084278
9,564181993697339392,722067


In [106]:
mpa_dict = mpa_dedup_df.set_index('ID').to_dict(orient = 'index')

In [107]:
mpa_cat_dict = {}

In [108]:
def getting_mpa_info(mpa_id, mpa_cat):
    row = mpa_cat[mpa_cat['SPECOBJID_1'] == mpa_id]    
    ans = row['BPTCLASS'][0]
    return ans

In [109]:
done_ids = list(mpa_cat_dict.keys())
for i in tqdm(list(mpa_dict.keys())):
    if i in done_ids:
        continue
    
    mpa_cat_dict[i] = getting_mpa_info(mpa_dict[i]['SPECOBJID_1'], mpa_jhu)

100%|██████████| 10/10 [00:06<00:00,  1.64it/s]


In [110]:
mpa_cat_df = pd.DataFrame.from_dict(mpa_cat_dict, orient = 'index').rename(columns = {'index' : 'ID', 0 : 'clsf'}).dropna()
mpa_cat_df

,clsf
241559,2
858969,-1
1121040,-1
1473567,-1
1156381,2
1431797,-1
576003,-1
1018703,-1
1084278,-1
722067,3


In [111]:
mpa_cat_df.clsf.value_counts()

-1    7
 2    2
 3    1
Name: clsf, dtype: int64

Spectral Definitions can be found here: https://home.strw.leidenuniv.nl/~jarle/SDSS/

In [112]:
def mpa_clsfs(clsf):
    if clsf == -1 or clsf == 0 or clsf == 3:
        return np.nan
    elif clsf == 1 or clsf == 2:
        return 'SFG'
    elif clsf == 4 or clsf == 5:
        return 'AGN'

In [113]:
mpa_clsf_df = (
    mpa_cat_df
    .assign(agn_clsf = mpa_cat_df.clsf.apply(lambda x: mpa_clsfs(x)))
    .reset_index()
    .rename(columns = {'index' : 'ID'})
    .drop(columns = 'clsf')
    .dropna()
)
mpa_clsf_df

,ID,agn_clsf
0,241559,SFG
4,1156381,SFG


### Putting it all Together!

chandra_agn_cat

vla_cat_df

clsf_df

smol_clsf_df

In [114]:
vla_cat_comb = vla_cat_df.reset_index().rename(columns = {'index' : 'ID', 'clsf' : 'agn_clsf' })

In [115]:
vla_cat_comb.drop_duplicates(subset='ID', keep='first').agn_clsf.value_counts()

SFG      2
HLAGN    2
Name: agn_clsf, dtype: int64

In [118]:
df_agn_cat = pd.concat([
    vla_cat_comb,
    smol_clsf_df, 
    chandra_cat_df.reset_index().rename(columns = {'index' : 'ID', 'clsf' : 'agn_clsf'}), 
    mpa_clsf_df])
# df_agn_cat = pd.concat([vla_cat_comb, smol_clsf_df])

In [119]:
len(df_agn_cat)

1362

In [120]:
df_agn_cat

,ID,agn_clsf
0,765389,SFG
1,715417,HLAGN
2,1485262,SFG
3,998663,HLAGN
0,888234,AGN
...,...,...
218,500953,AGN
219,410793,AGN
220,467138,AGN
0,241559,SFG


In [121]:
df_agn_cat.agn_clsf.value_counts()

SFG      828
AGN      532
HLAGN      2
Name: agn_clsf, dtype: int64

In [122]:
len(df_agn_cat.drop_duplicates(subset='ID', keep='first'))

1362

In [123]:
df_agn_dedup = df_agn_cat.drop_duplicates(subset='ID', keep='first')

In [124]:
df_agn_dedup.agn_clsf.value_counts()

SFG      828
AGN      532
HLAGN      2
Name: agn_clsf, dtype: int64

In [125]:
df_agn_dedup.to_csv(f'{combine_fold}/source-agn-cat.csv')